In [ ]:
import tensorflow as tf


### Consume the TFrecord data

*  https://www.tensorflow.org/programmers_guide/datasets
*  https://github.com/Tony607/Keras_catVSdog_tf_estimator/blob/master/keras_estimator_vgg16-cat_vs_dog-TFRecord.ipynb

In [ ]:
filenames = [ './librivox/guidetomen_%02d_rowland_64kb.mp3' % (i,) for i in [1,2,3]]
filenames

In [ ]:
steps_total, steps_leadin = 1024, 64
mel_bins, spectra_bins = 80, 1025

#batch_size, num_epochs = 8, 10

In [ ]:
def dataset_from_mp3(filenames, stub='train'):
    dataset = tf.data.TFRecordDataset([f.replace('.mp3', '_%s.tfrecords') % stub 
                                       for f in filenames])

    features = {
      "mel":          tf.FixedLenFeature([mel_bins*steps_total], tf.float32),
      "spectra_real": tf.FixedLenFeature([spectra_bins*steps_total], tf.float32),
      "spectra_imag": tf.FixedLenFeature([spectra_bins*steps_total], tf.float32),
    }
    
    def _parse_function(example_proto):
        parsed_features = tf.parse_single_example(example_proto, features)
        mel     = parsed_features["mel"]
        spectra = tf.stack( [parsed_features["spectra_real"], parsed_features["spectra_imag"], ] )
        return mel.reshape( (mel_bins, steps_total, 1) ), spectra.reshape( (spectra_bins, steps_total, 2) )

    dataset = dataset.map(_parse_function)
    return dataset

def input_fn_from(filenames, stub='train', batch_size=1, shuffle=False, repeats=1):
    dataset = dataset_from_mp3(filenames, stub=stub)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset_train.batch(batch_size).repeat(repeats)
    
    iterator_train = dataset.make_one_shot_iterator()    
    
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels    